In [149]:
#conflict
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
import pandas as pd
from pandas import DataFrame as df
from mpld3 import plugins, fig_to_html, save_html, fig_to_dict
from tqdm import tqdm
import numpy as np
import json
from tensorflow.keras.preprocessing.text import text_to_word_sequence
#character, setting
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models, similarities

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cacki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\cacki\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [150]:
text_input = """Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt """


In [151]:
def aiwriter(input_text):
    
    #데이터 1000의 문장을. 로 구분하여 리스테 넣어놓기
    #중간연사~~~ 삐리릭 삐삐~
    #결과값을 토대로 문장을 가져와서 출력하기
    
    documents = ["Some students have a background, identity, interest, or talent that is so meaningful they believe their application would be incomplete without it. If this sounds like you, then please share your story.",
                "The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?",
                "Reflect on a time when you questioned or challenged a belief or idea. What prompted your thinking? What was the outcome?",
                "Describe a problem you've solved or a problem you'd like to solve. It can be an intellectual challenge, a research query, an ethical dilemma - anything that is of personal importance, no matter the scale. Explain its significance to you and what steps you took or could be taken to identify a solution.",
                "Discuss an accomplishment, event, or realization that sparked a period of personal growth and a new understanding of yourself or others.",
                "Describe a topic, idea, or concept you find so engaging that it makes you lose all track of time. Why does it captivate you? What or who do you turn to when you want to learn more? ",
                "Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt, or one of your own design."]

    df = pd.DataFrame(documents, columns =['sentences'])
    
    # remove common words and tokenize them... 100ea. upto 1000ea
    stoplist = set('''for a of the and to in a about all also and as at be because but by
                    can come could day do even find first for from get give go have he her here him his how I if in into it its
                    just know like look make man many me more my new no not now of on one only or other our out people say see she
                    so some take tell than that the their them then there these they thing think this those time to two up use very
                    want way we well what when which who will with would year you your'''.split())

    texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

    # remove words those appear only once
    all_tokens = sum(texts, [])

    tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) ==1)
    texts = [[word for word in text if word not in tokens_once]
            for text in texts]
    dictionary = corpora.Dictionary(texts)#make dictionary  문장을 수치화 하여 사전으로 만듬

    dictionary.save('similerlist.dict')  # save as binary file at the dictionary at local directory 일단 사전을 만든다. 
    dictionary.save_as_text('similerlist_text.dict')  # save as text file at the local directory



    #input answer
    text_input = input_text #문장입력....
 
    new_vec = dictionary.doc2bow(text_input.lower().split()) # return "word-ID : Frequency of appearance"" 소문자, 빈칸구분, 빈도수 수치화 하여 
    corpus = [dictionary.doc2bow(text) for text in texts] #문장을 수치화하여 코퍼스 리스트로 만듬
    corpora.MmCorpus.serialize('similerlist.mm', corpus) # save corpus at local directory Matrix Market format으로 저장
    corpus = corpora.MmCorpus('similerlist.mm') # try to load the saved corpus from local 다시 로딩
    dictionary = corpora.Dictionary.load('similerlist.dict') # try to load saved dic.from local
    tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model 초기화
    corpus_tfidf = tfidf[corpus]  # 코퍼스 모델을 tfidf공간에 맵핑
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # LSI 초기화
    corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus
    topic = lsi.print_topics(2)
    lsi.save('model.lsi')  # save output model at local directory
    lsi = models.LsiModel.load('model.lsi') # try to load above saved model

    doc = text_input

    vec_bow = dictionary.doc2bow(doc.lower().split())  # put newly obtained document to existing dictionary object
    vec_lsi = lsi[vec_bow] # convert new document (henceforth, call it "query") to LSI space
    index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and indexize it
    index.save('similerlist.index') # save index object at local directory
    index = similarities.MatrixSimilarity.load('similerlist.index')
    sims = index[vec_lsi] # calculate degree of similarity of the query to existing corpus

    #print(list(enumerate(sims))) # output (document_number , document similarity)

    #sims = sorted(enumerate(sims), key=lambda item: -item[1])  # sort output object as per similarity ( largest similarity document comes first )
    #print(sims) # 가장 질문에 대한 답변이 적합한 순서대로 출력
    
    #df2 = pd.DataFrame(sims)
    #return df, df2

    df2 = pd.DataFrame(sims ,columns =['similarity'])
    df3 = pd.concat([df,df2], axis=1) #문장과 유사도 계산 결과를 데이터프레임으로 합치고
    df4 =  df3.sort_values(by=['similarity'], axis=0, ascending=False)
    return df4[:3] #가장 비슷한 문장 3개를 보여줌

In [152]:
aiwriter(text_input) # 결과에서 6,3,4를 유사한 문장으로 추천해 줘야 함

,sentences,similarity
6,Share an essay on any topic of your choice. It...,1.000000
3,Describe a problem you've solved or a problem ...,0.982750
4,"Discuss an accomplishment, event, or realizati...",0.974974
